# Convert  Object Store datapoints - Airbus NationWide - into Visual search Maps

In [1]:
import pandas
print(pandas.__version__)

0.24.2


In [2]:
import folium
print(folium.__version__)

0.14.0


In [3]:
import numpy as np 
import pandas as pd 
import folium
from folium.plugins import Search
from functools import reduce
import math
# No need since 28Apr23 - Lat/Lon provided by Airbus
# Convert BNG to LAT/LON
#from bng_latlon import OSGB36toWGS84
#
input_coord_csv_file='data/output/NBS_Portfolio_Bristol_latlon_230216_csv/part-00000-5c3ba62f-e549-448a-bfa0-6d70538207c5-c000.csv'
df_nationwide_loanbook = pd.read_csv(input_coord_csv_file, header = 0 )


In [4]:
# df_nationwide_loanbook.describe()
df_nationwide_loanbook.dtypes

ID                          int64
UPRN                        int64
POSTCODE_FLAG               int64
EASTING                   float64
NORTHING                  float64
RHIGHSCORE                  int64
RHIGHSCORE_ADJ              int64
CHIGHSCORE                  int64
CHIGHSCORE_ADJ              int64
SWHIGHSCORE                 int64
SWHIGHSCORE_ADJ             int64
GWHIGHSCORE                 int64
COMBINEDSCORE               int64
COMBINEDSCORE_ADJ           int64
R_RCP26_2050_UD           float64
R_RCP26_2050_D            float64
S_RCP26_2050_UD           float64
S_RCP26_2050_D            float64
C_RCP26_2050_UD           float64
C_RCP26_2050_D            float64
COMB_RCP26_2050_UD        float64
COMB_RCP26_2050_D         float64
R_RCP45_2050_UD           float64
R_RCP45_2050_D            float64
S_RCP45_2050_UD           float64
S_RCP45_2050_D            float64
C_RCP45_2050_UD           float64
C_RCP45_2050_D            float64
COMB_RCP45_2050_UD        float64
COMB_RCP45_205

In [5]:
# Lambda function apply convertion CURRENTLOAN and LTV to INITIAL_BORROW_CAPITAL
#CURRENTLOAN                         float64
#CURRENTLTV                     float64

vars_ln=['CURRENTLOAN','CURRENTLTV']
cuurlnamt='CURRENTLOAN'
currltv='CURRENTLTV'
df_nationwide_loanbook['INITIAL_BORROW_CAPITAL'] = df_nationwide_loanbook[vars_ln].apply(lambda x :  x[cuurlnamt]*1/x[currltv] ,axis=1)


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_nationwide_loanbook.dtypes)



ID                                    int64
UPRN                                  int64
POSTCODE_FLAG                         int64
EASTING                             float64
NORTHING                            float64
RHIGHSCORE                            int64
RHIGHSCORE_ADJ                        int64
CHIGHSCORE                            int64
CHIGHSCORE_ADJ                        int64
SWHIGHSCORE                           int64
SWHIGHSCORE_ADJ                       int64
GWHIGHSCORE                           int64
COMBINEDSCORE                         int64
COMBINEDSCORE_ADJ                     int64
R_RCP26_2050_UD                     float64
R_RCP26_2050_D                      float64
S_RCP26_2050_UD                     float64
S_RCP26_2050_D                      float64
C_RCP26_2050_UD                     float64
C_RCP26_2050_D                      float64
COMB_RCP26_2050_UD                  float64
COMB_RCP26_2050_D                   float64
R_RCP45_2050_UD                 

In [7]:
df_nationwide_loanbook.describe

<bound method NDFrame.describe of           ID          UPRN  POSTCODE_FLAG    EASTING   NORTHING  RHIGHSCORE  \
0          1           147              0  358754.72  179718.88           0   
1          2           148              0  358946.00  179716.00           0   
2          7           155              0  358974.00  179709.00           0   
3          8           157              0  358763.00  179704.00           0   
4          9           172              0  358792.00  179695.00           0   
5          3           180              0  356788.00  179691.00          18   
6          4           183              0  356598.00  179689.00           0   
7          5           186              0  358874.00  179687.00           0   
8          6           198              0  358725.00  179680.00           0   
9         10           223              0  358888.00  179670.00           0   
10        11           233              0  358974.00  179667.00           0   
11        12      

In [8]:
# BUILT_FORM
df_nationwide_loanbook_play = df_nationwide_loanbook.groupby('BUILT_FORM').BUILT_FORM.count()
df_nationwide_loanbook_play

BUILT_FORM
Detached                2274
Enclosed End-Terrace      87
Enclosed Mid-Terrace      44
End-Terrace             2271
Mid-Terrace             4923
NO DATA!                 218
Semi-Detached           4863
Name: BUILT_FORM, dtype: int64

In [9]:
# LOCAL_AUTHORITY_LABEL 
df_nationwide_loanbook_play = df_nationwide_loanbook.groupby('LOCAL_AUTHORITY_LABEL').LOCAL_AUTHORITY_LABEL.count()
df_nationwide_loanbook_play

LOCAL_AUTHORITY_LABEL
Bath and North East Somerset    1747
Bristol, City of                7654
Camden                             1
North Somerset                   280
South Gloucestershire           4986
Tower Hamlets                      1
Wiltshire                         18
Name: LOCAL_AUTHORITY_LABEL, dtype: int64

In [10]:
# Count and map more than one report per house Key UPRN
attr = 'UPRN'
min_records = 2
df2 = df_nationwide_loanbook.groupby(attr).filter(lambda x: len(x) >= min_records)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df2.dtypes)
    print(df2)

ID                                    int64
UPRN                                  int64
POSTCODE_FLAG                         int64
EASTING                             float64
NORTHING                            float64
RHIGHSCORE                            int64
RHIGHSCORE_ADJ                        int64
CHIGHSCORE                            int64
CHIGHSCORE_ADJ                        int64
SWHIGHSCORE                           int64
SWHIGHSCORE_ADJ                       int64
GWHIGHSCORE                           int64
COMBINEDSCORE                         int64
COMBINEDSCORE_ADJ                     int64
R_RCP26_2050_UD                     float64
R_RCP26_2050_D                      float64
S_RCP26_2050_UD                     float64
S_RCP26_2050_D                      float64
C_RCP26_2050_UD                     float64
C_RCP26_2050_D                      float64
COMB_RCP26_2050_UD                  float64
COMB_RCP26_2050_D                   float64
R_RCP45_2050_UD                 

In [11]:
df_nationwide_loanbook.groupby('CURRENTLOAN').CURRENTLOAN.count()

CURRENTLOAN
505.64       1
523.00       1
525.09       1
529.52       1
533.58       1
538.18       1
542.24       1
543.30       1
564.14       1
565.02       1
577.81       1
582.92       1
591.54       1
591.80       1
609.22       1
618.88       1
619.21       1
622.12       1
631.56       1
634.01       1
660.22       1
664.97       1
698.82       1
702.91       1
739.54       1
789.34       1
794.28       1
817.05       1
844.05       1
876.99       1
            ..
545305.57    1
546324.34    1
549697.62    1
553921.83    1
553992.79    1
556164.70    1
557313.79    1
564391.99    1
571426.78    1
576445.72    1
587421.52    1
597151.98    1
600432.98    1
607466.46    1
634509.38    1
638887.09    1
649220.46    1
657828.36    1
664388.47    1
686458.89    1
694192.54    1
713449.98    1
722228.97    1
738026.27    1
742493.79    1
742671.05    1
753046.37    1
792920.56    1
837784.96    1
871039.06    1
Name: CURRENTLOAN, Length: 18078, dtype: int64

In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_nationwide_loanbook.head(12))

    ID  UPRN  POSTCODE_FLAG    EASTING   NORTHING  RHIGHSCORE  RHIGHSCORE_ADJ  \
0    1   147              0  358754.72  179718.88           0               0   
1    2   148              0  358946.00  179716.00           0               0   
2    7   155              0  358974.00  179709.00           0               0   
3    8   157              0  358763.00  179704.00           0               0   
4    9   172              0  358792.00  179695.00           0               0   
5    3   180              0  356788.00  179691.00          18               5   
6    4   183              0  356598.00  179689.00           0               0   
7    5   186              0  358874.00  179687.00           0               0   
8    6   198              0  358725.00  179680.00           0               0   
9   10   223              0  358888.00  179670.00           0               0   
10  11   233              0  358974.00  179667.00           0               0   
11  12   292              0 

In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_nationwide_loanbook.tail(20))

          ID          UPRN  POSTCODE_FLAG    EASTING   NORTHING  RHIGHSCORE  \
18065  18056  100121180318              0  357488.00  162920.00          18   
18066  18057  100121180369              0  357503.00  162915.00          20   
18067  18044  100121180376              0  357603.18  162866.30           1   
18068  18058  100121180399              0  357682.00  163408.00           0   
18069  18050  100121180470              0  356297.00  161612.00           0   
18070  18071  100121396620              0  376050.00  166305.00           0   
18071  18072  100121396679              0  374926.00  165642.00           0   
18072  18073  100121397216              0  375612.00  165184.00           0   
18073  18080  200001299301              0  379793.00  160537.00           0   
18074  18078  200001299431              0  378215.00  161733.00           0   
18075  18079  200001300211              0  379865.00  160843.00           0   
18076  18074  200001300253              0  379842.00

In [14]:
# display all distinct CURRENT_ENERGY_RATING values
column_list = ['CURRENT_ENERGY_RATING']
for column_name in column_list:
      print(df_nationwide_loanbook[column_name].unique())

['E' 'D' nan 'C' 'B' 'F' 'G' 'A']


In [15]:
# display all distinct COMBINEDSCORE_ADJ values
column_list = ['COMBINEDSCORE_ADJ']
for column_name in column_list:
      print(df_nationwide_loanbook[column_name].unique())

[ 0  2  5  8  1  3 23  7 10  6 22 11  9  4 12 26 28 17 18 21 15 30 20 24
 27 25 19 34 37 13 14 16 32 42 35]


In [16]:
# display all distinct COMBINEDSCORE values
column_list = ['COMBINEDSCORE']
for column_name in column_list:
      print(df_nationwide_loanbook[column_name].unique())

[ 0  2 18  5  8  1  3 23  7 21 20 12 17 25 22  4 10 28 16 14  9  6 26 24
 15 30 27 19 35 42 45 13 33 48 31 38 40 37 29 11 39 43 34 32 36 44 47]


In [17]:
# display COMB_RCP26_2050_UD - Climate Change Combined Flood Score (undefended) for RCP 2.6 for 2050

column_list = ['COMB_RCP26_2050_UD']
for column_name in column_list:
      print(df_nationwide_loanbook[column_name].unique())


[ 0. 18.  3.  5.  1.  2.  8. 23.  9. 20.  4. 17. 22. 25. 28.  6. 14. 24.
 16. 19. 13. 10. 30. 12. 27. 21. 26. 15. 11. 29. 40. 42. 48. 43. 34. 35.
 38. 37. 31. 32. 39. 45. 52. 47. 50. 41. nan 44.  7.]


In [18]:
# display COMB_RCP26_2050_UD - Climate Change Combined Flood Score (defended) for RCP 2.6 for 2050

column_list = ['COMB_RCP26_2050_D']
for column_name in column_list:
      print(df_nationwide_loanbook[column_name].unique())

[ 0. 18.  3.  5.  1.  2.  8. 23.  9. 20.  4. 10. 12. 28.  7. 11. 15. 14.
 13. 19. 30. 24. 25. 27. 22. 17. 21. 26. 16. 37. 45. 32.  6. 35. 29. nan
 50.]


In [19]:
# CONTINUE WITH MAP LeafLeat Folium map generation of HTML

In [20]:
# Increase display in Jupyter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [21]:
df_nationwide_loanbook.shape

(18085, 201)

In [22]:
lat= 51.4467729
lon= -2.5985602


In [23]:
#Color codes for EPC rating scale
colors_epc={
    np.nan:'#deebf6',
    'A':'#00a54f',
    'B':'#4cb848',
    'C':'#bed630',
    'D':'#fff101',
    'E':'#fcb814',
    'F':'#f36e21',
    'G':'#ee1d23',
}

In [24]:
#Color codes for flood rating scale
colours_flood={
    'no-colour':'#deebf6',
    'green':'#a5d45d',
    'amber':'#ecc63a',
    'red':'#d92424',
    'black-1':'#919191',
    'black-2':'#242424',
    'purple':'#9370db',
}

# Function to convert number into string
# Switcher is dictionary data type here
### Planned for COMBINEDSCORE/COMBINEDSCORE_ADJ -  Combined Floodability undefended/defended - score
def FloodRatingSwitch(undefend_sum,defend_sum):
    value_search = undefend_sum if (undefend_sum > defend_sum) else defend_sum
    # print('value_search_='+str(value_search))
    list_no_colour = [0]
    list_green = [1,2,3,4,5,6,7,8,9,10]
    list_amber = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
    list_red = [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]
    list_black1 = [45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
    list_black2 = [61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100]
    # Evaluate from List
    if value_search in list_no_colour:
        return 'no-colour'
    elif value_search in list_green:
        return 'green'
    elif value_search in list_amber:
        return 'amber'
    elif value_search in list_red:
        return 'red'
    elif value_search in list_black1:
        return 'black-1'
    elif value_search in list_black2:
        return 'black-2'
    else:
        return 'purple' #ERROR out-of-bounds
    
    # get() method of dictionary data type returns
    # value of passed argument if it is present
    # in dictionary otherwise second argument will
    # be assigned as default value of passed argument
    # return switcher.get(value_search, 'purple') # 'purple':'#9370db', ERROR
    
## Function to Define Ratio Value between 0 ( 0.001) and 100 as %
def FloodRating_Ration_0_100_Value(undefend_sum,defend_sum):
    value_search = undefend_sum if (undefend_sum > defend_sum) else defend_sum
    # augment in 40% for colour grading on map
    ratio_0_100 = (value_search/100)*1.40 + 0.001
    ratio_0_100 = round( ratio_0_100, 4)
    return ratio_0_100

In [25]:
# PROPERTY_TYPE
df_nationwide_loanbook_play = df_nationwide_loanbook.groupby('PROPERTY_TYPE').CONSTRUCTION_AGE_BAND.count()
df_nationwide_loanbook_play

PROPERTY_TYPE
Bungalow        259
Flat           1815
House         12295
Maisonette      154
Name: CONSTRUCTION_AGE_BAND, dtype: int64

In [26]:
# CONSTRUCTION_AGE_BAND
df_nationwide_loanbook_play = df_nationwide_loanbook.groupby('CONSTRUCTION_AGE_BAND').CONSTRUCTION_AGE_BAND.count()
df_nationwide_loanbook_play

CONSTRUCTION_AGE_BAND
2017                                 12
2018                                  6
2019                                 36
2020                                 48
2021                                122
2022                                 67
England and Wales: 1900-1929       1869
England and Wales: 1930-1949       2355
England and Wales: 1950-1966       2329
England and Wales: 1967-1975       1377
England and Wales: 1976-1982        537
England and Wales: 1983-1990        669
England and Wales: 1991-1995        391
England and Wales: 1996-2002        573
England and Wales: 2003-2006        414
England and Wales: 2007 onwards     202
England and Wales: 2007-2011         76
England and Wales: 2012 onwards      13
England and Wales: before 1900     1807
INVALID!                             13
NO DATA!                           1607
Name: CONSTRUCTION_AGE_BAND, dtype: int64

In [27]:
# BUILT_FORM
df_nationwide_loanbook_play = df_nationwide_loanbook.groupby('BUILT_FORM').ROOF_DESCRIPTION.count()
df_nationwide_loanbook_play

BUILT_FORM
Detached                2274
Enclosed End-Terrace      87
Enclosed Mid-Terrace      44
End-Terrace             2271
Mid-Terrace             4923
NO DATA!                 218
Semi-Detached           4861
Name: ROOF_DESCRIPTION, dtype: int64

In [28]:
# ROOF_DESCRIPTION
df_nationwide_loanbook_play = df_nationwide_loanbook.groupby('ROOF_DESCRIPTION').ROOF_DESCRIPTION.count()
df_nationwide_loanbook_play

ROOF_DESCRIPTION
(another dwelling above)                           801
(other premises above)                             308
(other premises above)|(eiddo arall uwchben)         4
Average thermal transmittance 0 W/m?K                9
Average thermal transmittance 0.00 W/m&#0178;K       2
Average thermal transmittance 0.04 W/m?K             1
Average thermal transmittance 0.07 W/m-�K            2
Average thermal transmittance 0.07 W/m?K             2
Average thermal transmittance 0.08 W/m-�K           20
Average thermal transmittance 0.09 W/m-�K           49
Average thermal transmittance 0.09 W/m?K             3
Average thermal transmittance 0.1 W/m-�K            47
Average thermal transmittance 0.1 W/m?K              1
Average thermal transmittance 0.10 W/m+�-�K          2
Average thermal transmittance 0.10 W/m-�K          104
Average thermal transmittance 0.10 W/m2K             1
Average thermal transmittance 0.10 W/m??K            1
Average thermal transmittance 0.10 W/m?K        

In [29]:
# Creating the popup labels for energy rating data 

def popup_html_epc(row):
    i = row
    uprn_id= df_nationwide_loanbook['UPRN'].iloc[i] 
    address= 'Property_type='+str(df_nationwide_loanbook['PROPERTY_TYPE'].iloc[i])+'. '+str(df_nationwide_loanbook['BUILT_FORM'].iloc[i])+'. '+  str( df_nationwide_loanbook['POSTCODE_FLAG'].iloc[i] ) +'. Easting/Norting='+ str( df_nationwide_loanbook['EASTING'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['NORTHING'].iloc[i] ) +'. Latitude/Longitude='+ str( df_nationwide_loanbook['LATITUDE'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['LONGITUDE'].iloc[i] )
    Epc_Rating = df_nationwide_loanbook['CURRENT_ENERGY_RATING'].iloc[i]
    Contruction_Age = df_nationwide_loanbook['CONSTRUCTION_AGE_BAND'].iloc[i]
    Glazed_Type = df_nationwide_loanbook['GLAZED_TYPE'].iloc[i]
    Epc_Energy_Cons_Kwh = df_nationwide_loanbook['ENERGY_CONSUMPTION_CURRENT'].iloc[i]
    Co2_Emission = df_nationwide_loanbook['CO2_EMISSIONS_CURRENT'].iloc[i]
    
    Hotwater = df_nationwide_loanbook['HOTWATER_DESCRIPTION'].iloc[i]
    Hotwater_energy_eff = df_nationwide_loanbook['HOT_WATER_ENERGY_EFF'].iloc[i]
    Hotwater_env_eff = df_nationwide_loanbook['HOT_WATER_ENV_EFF'].iloc[i]

    Floor = df_nationwide_loanbook['FLOOR_DESCRIPTION'].iloc[i]
    Floor_energy_eff = df_nationwide_loanbook['FLOOR_ENERGY_EFF'].iloc[i]
    Floor_env_eff = df_nationwide_loanbook['FLOOR_ENV_EFF'].iloc[i]

    Windows = df_nationwide_loanbook['WINDOWS_DESCRIPTION'].iloc[i]
    Windows_energy_eff = df_nationwide_loanbook['WINDOWS_ENERGY_EFF'].iloc[i]
    Windows_env_eff = df_nationwide_loanbook['WINDOWS_ENV_EFF'].iloc[i]

    Walls = df_nationwide_loanbook['WALLS_DESCRIPTION'].iloc[i]
    Walls_energy_eff = df_nationwide_loanbook['WALLS_ENERGY_EFF'].iloc[i]
    Walls_env_eff = df_nationwide_loanbook['WALLS_ENV_EFF'].iloc[i]

    Roof = df_nationwide_loanbook['ROOF_DESCRIPTION'].iloc[i]
    Roof_energy_eff = df_nationwide_loanbook['ROOF_ENERGY_EFF'].iloc[i]
    Roof_env_eff = df_nationwide_loanbook['ROOF_ENV_EFF'].iloc[i]

    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">UPRN ID:{}</h4>""".format(uprn_id) + """</center>
<center> <table style="height: 90px; width: 600px;">
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property </span></td>
<td style="border: 1px solid white; width: 200px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Epc Rating </span></td>
<td style="border: 1px solid white; width: 200px;background-color: """+ right_col_color +""";">{}</td>""".format(Epc_Rating) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Epc Energy Consumption Kwh </span></td>
<td style="border: 1px solid white; width: 200px;background-color: """+ right_col_color +""";">{} kWh/m\u00b2/year</td>""".format(Epc_Energy_Cons_Kwh) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">CO\u2082 Emission Current</span></td>
<td style="border: 1px solid white; width: 200px;background-color: """+ right_col_color +""";">{}</td>""".format(Co2_Emission) + """
</tr>
</tbody>
</table></center>


<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Construction and Glazing</h5>
<center><table style="height: 50px; width: 600px;">
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Contruction Age</span></td>
<td style="border: 1px solid white; width: 200px;background-color: """+ right_col_color +""";">{}</td>""".format(Contruction_Age) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Glazing</span></td>
<td style="border: 1px solid white; width: 200px;background-color: """+ right_col_color +""";">{}</td>""".format(Glazed_Type) + """
</tr>
</tbody>
</table></center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold"> EPC Energy Rating and Consumption</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Current </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Potential </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> EPC Energy Rating and Consumption </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} ({} kWh/m\u00b2/year)</td>""".format(df_nationwide_loanbook['CURRENT_ENERGY_RATING'].iloc[i],df_nationwide_loanbook['ENERGY_CONSUMPTION_CURRENT'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} ({} kWh/m\u00b2/year)</td>""".format(df_nationwide_loanbook['POTENTIAL_ENERGY_RATING'].iloc[i],df_nationwide_loanbook['ENERGY_CONSUMPTION_POTENTIAL'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold"> Property CO\u2082 Emissions</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Current </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Potential </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> CO\u2082 Emissions </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} CO\u2082 Tonnes/year </td>""".format(df_nationwide_loanbook['CO2_EMISSIONS_CURRENT'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} CO\u2082 Tonnes/year </td>""".format(df_nationwide_loanbook['CO2_EMISSIONS_POTENTIAL'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold"> Energy Efficiency</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Current </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Potential </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Energy Efficiency </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} Based on cost of energy, i.e. heating, water and lighting [in kWh/year] mult. by fuel costs. (£/m²/year cost is derived from kWh). </td>""".format(df_nationwide_loanbook['CURRENT_ENERGY_EFFICIENCY'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} Based on cost of energy, i.e. heating, water and lighting [in kWh/year] mult. by fuel costs. (£/m²/year cost is derived from kWh). </td>""".format(df_nationwide_loanbook['POTENTIAL_ENERGY_EFFICIENCY'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Hotwater, Floor, Windows, Wall Efficiency Information</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Decription </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Energy Efficiency</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Environment Efficiency</span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Hotwater</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(Hotwater) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(Hotwater_energy_eff) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(Hotwater_env_eff) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Floor</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(Floor) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(Floor_energy_eff) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(Floor_env_eff) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Windows</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(Windows) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(Windows_energy_eff) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(Windows_env_eff) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Walls</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(Walls) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(Walls_energy_eff) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(Walls_env_eff) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Roof</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(Roof) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(Roof_energy_eff) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(Roof_env_eff) + """
</tr>
</tbody>
</table> </center>


<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Loan Details</h5>
<center> <table style="height: 50px; width:600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">UPRN </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan Balance</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan to Value LTV</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Initial Borrow Capital</span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['UPRN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">£{}</td>""".format(df_nationwide_loanbook['CURRENTLOAN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}%</td>""".format(round(100*df_nationwide_loanbook['CURRENTLTV'].iloc[i],4)) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">£{}</td>""".format(round(df_nationwide_loanbook['INITIAL_BORROW_CAPITAL'].iloc[i],2)) + """
</tr>
</tbody>
</table> </center>


</html>
"""
    return html

In [30]:
# display COMB_RCP26_2050_UD - Climate Change Combined Flood Score (defended) for RCP 2.6 for 2050

column_list = ['SOP_RIVER_VALUE']
for column_name in column_list:
      print(df_nationwide_loanbook[column_name].unique())

[  0 100   5  25  50  10 500 200  75 150]


In [31]:
# Creating the popup labels for flood rating data

def popup_html_flood(row):
    i = row
    uprn_id=df_nationwide_loanbook['UPRN'].iloc[i] 
    address= 'Property_type='+str(df_nationwide_loanbook['PROPERTY_TYPE'].iloc[i])+'. '+str(df_nationwide_loanbook['BUILT_FORM'].iloc[i])+'. '+  str( df_nationwide_loanbook['POSTCODE_FLAG'].iloc[i] ) +'. Easting/Norting='+ str( df_nationwide_loanbook['EASTING'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['NORTHING'].iloc[i] ) +'. Latitude/Longitude='+ str( df_nationwide_loanbook['LATITUDE'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['LONGITUDE'].iloc[i] )
    river_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i]
    coastal_flooding_first = df_nationwide_loanbook['CHIGHSCORE'].iloc[i]
    surf_water_flooding_first = df_nationwide_loanbook['SWHIGHSCORE'].iloc[i]
    combined_scoring_flooding_first = df_nationwide_loanbook['COMBINEDSCORE'].iloc[i]
    river_value = df_nationwide_loanbook['SOP_RIVER_VALUE'].iloc[i]
    water_surface_value = df_nationwide_loanbook['SOP_SURFACE_WATER_VALUE'].iloc[i]
    canal_failure = df_nationwide_loanbook['CANAL_FAILURE_VALUE'].iloc[i]
    dam_break = df_nationwide_loanbook['DAM_BREAK_VALUE'].iloc[i]
    ground_height = df_nationwide_loanbook['UNFLOOD_VALUE'].iloc[i]
    peat_surface = df_nationwide_loanbook['NPD_PEAT'].iloc[i]
    sand_soil = df_nationwide_loanbook['NPD_SAND'].iloc[i]
    soft_soil = df_nationwide_loanbook['NPD_SOFT'].iloc[i]
    silt_soil = df_nationwide_loanbook['NPD_SILT'].iloc[i]
    
    
    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">UPRN ID:{}</h4>""".format(uprn_id) + """</center>
<center> <table style="height: 90px; width: 500px;">
<tbody>
<tr>
<td style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Highest score from the Defended river scores  </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(river_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Highest score from the Defended coastal scores </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(coastal_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the Defended surface water scores </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(surf_water_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Combined Sum of all Scores - Defended River, Coastal and Surface Water</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(combined_scoring_flooding_first) + """
</tr>
</tbody>
</table>
</center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Loan Details</h5>
<center> <table style="height: 50px; width:600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">UPRN </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan Balance</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan to Value LTV</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Initial Borrow Capital</span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['UPRN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">£{}</td>""".format(df_nationwide_loanbook['CURRENTLOAN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}%</td>""".format(round(100*df_nationwide_loanbook['CURRENTLTV'].iloc[i],4)) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">£{}</td>""".format(round(df_nationwide_loanbook['INITIAL_BORROW_CAPITAL'].iloc[i],2)) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Floodability score</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Floodability Type </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">River</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['RHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Coastal</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['CHIGHSCORE'].iloc[i])  + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['CHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Surface Water</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['SWHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['SWHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Overall score</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";"><b>{} </b></td>""".format(df_nationwide_loanbook['COMBINEDSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";"><b>{} </b></td>""".format(df_nationwide_loanbook['COMBINEDSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Ground Water</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y highest </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the ground water scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['GWHIGHSCORE'].iloc[i]) + """
</tr>
</tbody>
</table> </center>


<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Dam and Canal Failure</h5>
<center> <table style="height: 50px; width: 600px;">
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Risk to flood following canal failure </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(canal_failure) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Risk to flood following dam break </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(dam_break) + """
</tr>
</tbody>
</table> </center>
<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Ground Height Information</h5>
<center> <table style="height: 50px; width: 600px;">
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> How high the ground is above flooding </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(ground_height) + """
</tr>
</tbody>
</table> </center>
<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Fixed defences and standard of protection</h5>
<center> <table style="height: 50px; width: 600px;">
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Any fixed river flood defences and the standard of protection identified </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(river_value) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Any fixed surface water flood defences and the standard of protection identified </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(water_surface_value) + """
</tr>
</tbody>
</table> </center>
<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Soil Related Information</h5>
<center> <table style="height: 50px; width: 600px;">
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Risk of peat relted subsidence </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(peat_surface) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Risk of sand related subsidence </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(sand_soil) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Risk of silt related subsidence </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(silt_soil) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Risk of soft soil related subsidence </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(soft_soil) + """
</tr>
</tbody>
</table> </center>
</html>
""" 
    return html

In [32]:
# Creating the popup labels for Airbus/NWide  loan data

def popup_html_index(row):
    i = row
    uprn_id=df_nationwide_loanbook['UPRN'].iloc[i] 
    address= 'Property_type='+str(df_nationwide_loanbook['PROPERTY_TYPE'].iloc[i])+'. '+str(df_nationwide_loanbook['BUILT_FORM'].iloc[i])+'. '+  str( df_nationwide_loanbook['POSTCODE_FLAG'].iloc[i] ) +'. Easting/Norting='+ str( df_nationwide_loanbook['EASTING'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['NORTHING'].iloc[i] ) +'. Latitude/Longitude='+ str( df_nationwide_loanbook['LATITUDE'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['LONGITUDE'].iloc[i] )
    river_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i]
    coastal_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i] 
    surf_water_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i]
    
    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px"> UPRN :{}</h4>""".format(uprn_id) + """</center>
<center> <table style="height: 90px; width: 600px;">
<tbody>
<tr>
<td style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current EPC Energy Rating and Consumption</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} ({} kWh/m\u00b2/year)</td>""".format(df_nationwide_loanbook['CURRENT_ENERGY_RATING'].iloc[i],df_nationwide_loanbook['ENERGY_CONSUMPTION_CURRENT'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Potential EPC Energy Rating and Consumption</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} ({} kWh/m\u00b2/year)</td>""".format(df_nationwide_loanbook['POTENTIAL_ENERGY_RATING'].iloc[i],df_nationwide_loanbook['ENERGY_CONSUMPTION_POTENTIAL'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current CO\u2082 Emissions</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} CO\u2082 Tonnes/year </td>""".format(df_nationwide_loanbook['CO2_EMISSIONS_CURRENT'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Potential CO\u2082 Emissions</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} CO\u2082 Tonnes/year </td>""".format(df_nationwide_loanbook['CO2_EMISSIONS_POTENTIAL'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Energy Efficiency</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}% Based on cost of energy, i.e. heating, water and lighting [in kWh/year] mult. by fuel costs. (£/m²/year cost is derived from kWh). </td>""".format(df_nationwide_loanbook['CURRENT_ENERGY_EFFICIENCY'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Potential Energy Efficiency</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}% Based on cost of energy, i.e. heating, water and lighting [in kWh/year] mult. by fuel costs. (£/m²/year cost is derived from kWh). </td>""".format(df_nationwide_loanbook['POTENTIAL_ENERGY_EFFICIENCY'].iloc[i]) + """
</tr>
</tbody>
</table>
</center>


<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Loan Details</h5>
<center> <table style="height: 50px; width:600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">UPRN </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan Balance</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan to Value LTV</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Initial Borrow Capital</span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['UPRN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">£{}</td>""".format(df_nationwide_loanbook['CURRENTLOAN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}%</td>""".format(round(100*df_nationwide_loanbook['CURRENTLTV'].iloc[i],4)) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">£{}</td>""".format(round(df_nationwide_loanbook['INITIAL_BORROW_CAPITAL'].iloc[i],2)) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Floodability score</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Floodability Type </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">River</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['RHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Coastal</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['CHIGHSCORE'].iloc[i])  + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['CHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Surface Water</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['SWHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} </td>""".format(df_nationwide_loanbook['SWHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Overall score</span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";"><b>{} </b></td>""".format(df_nationwide_loanbook['COMBINEDSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";"><b>{} </b></td>""".format(df_nationwide_loanbook['COMBINEDSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

</html>
""" 
    return html

In [33]:
#Creating lagends

def add_legend(maps, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_label = dict(zip(labels, colors))
    
    legends = ""     
    for label, color in color_label.items():
        legends += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='legend' class='legend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legends}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var Layout = (function() {{
                    var flag = false;
                    return function() {{
                        if (!flag) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!flag)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          flag = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        Layout()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .legend {
        z-index:9999;
        float:right;
        background-color: rgb(255, 255, 255);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .legend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .legend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .legend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .legend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .legend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .legend a {
        color: #777;
        }
    </style>
    """

    maps.get_root().header.add_child(folium.Element(script + css))

    return maps

# Display - NationWide & Loanbook  search by index ecad_id - UK Bristol County

In [34]:
#Plotting Nwide_loan data - index search

bristol_airbus_nwide_loanbook=folium.Map(location=[lat,lon],zoom_start=12)
from folium import plugins
cluster=plugins.MarkerCluster().add_to(bristol_airbus_nwide_loanbook)

for d in df_nationwide_loanbook.iterrows(): 
        html = popup_html_index(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=600)
        folium.Marker(location=[d[1]['LATITUDE'], d[1]['LONGITUDE']], popup=popup,name=d[1]["UPRN"]).add_to(cluster)
Search(cluster,search_label='name',placeholder='Search for UPRN ID').add_to(bristol_airbus_nwide_loanbook)

loc = ' Geo-Location Map - Search by UPRN - Nationwide Mortgages '
title_html = '''
             <h2 align="center" style="font-size:24px"><b>{}</b></h2>
             '''.format(loc) 
bristol_airbus_nwide_loanbook = bristol_airbus_nwide_loanbook.get_root().html.add_child(folium.Element(title_html))

bristol_airbus_nwide_loanbook.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Loanbook_IndexSearch.html')

# Display Optional
# bristol_airbus_nwide_loanbook

In [35]:
#Plotting EPC_rating data

bristol_epc=folium.Map(location=[lat,lon],zoom_start=14)

for d in df_nationwide_loanbook.iterrows():
        html = popup_html_epc(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        folium.CircleMarker(
                    [d[1]['LATITUDE'], d[1]['LONGITUDE']],
                    radius=6,
                    color=colors_epc[d[1]["CURRENT_ENERGY_RATING"]],
                    fill=True,
                    fill_color=colors_epc[d[1]["CURRENT_ENERGY_RATING"]],
                    fill_opacity=0.7,
                    popup=popup
            ).add_to(bristol_epc)

bristol_epc = add_legend(bristol_epc, 'Building Energy Rating', colors = list(colors_epc.values()), labels = list(colors_epc.keys()))


loc = ' Geo-Location Map - EPC Ratings - Nationwide Mortgages '
title_html = '''
             <h2 align="center" style="font-size:24px"><b>{}</b></h2>
             '''.format(loc) 
bristol_epc = bristol_epc.get_root().html.add_child(folium.Element(title_html))


bristol_epc.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Energy_Rating.html')

# Display Optional
# bristol_epc

In [36]:
#Plotting flood_rating data

bristol_flood=folium.Map(location=[lat,lon],zoom_start=14)

for d in df_nationwide_loanbook.iterrows():
        html = popup_html_flood(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        colour_d1point = FloodRatingSwitch(d[1]['COMBINEDSCORE_ADJ'], d[1]['COMBINEDSCORE'])
        folium.CircleMarker(
                    [d[1]['LATITUDE'], d[1]['LONGITUDE']],
                    radius=6,
                    color=colours_flood[colour_d1point],
                    fill=True,
                    fill_color=colours_flood[colour_d1point],
                    fill_opacity=0.7,
                    popup=popup
            ).add_to(bristol_flood)

bristol_flood = add_legend(bristol_flood, 'Likelihood of flooding', colors = list(colours_flood.values()), labels = ('Very Low', 'Low', 'Moderate', 'Moderate to High', 'High', 'Very High', 'ERROR'))

loc = ' Geo-Location Map - Floodability - Nationwide Mortgages '
title_html = '''
             <h2 align="center" style="font-size:24px"><b>{}</b></h2>
             '''.format(loc) 
bristol_flood = bristol_flood.get_root().html.add_child(folium.Element(title_html))


bristol_flood.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Flood_Rating.html')

# Display Optional 
#bristol_flood

In [37]:
## HeatMap Example
#### Follow this example of how to create a HEATMAP : https://wellsr.com/python/plotting-geographical-heatmaps-with-python-folium-module/ . 

from folium.plugins import HeatMap
import numpy

bristol_cluster = folium.Map(location=[lat,lon],zoom_start=17)

# HeatMap array of values
array_heatmap = []
for d in df_nationwide_loanbook.iterrows(): 
         score_all_weight_0_100 = FloodRating_Ration_0_100_Value(d[1]['COMBINEDSCORE_ADJ'], d[1]['COMBINEDSCORE'])
         array_val = [[ d[1]['LATITUDE'], d[1]['LONGITUDE'], score_all_weight_0_100 ]]
         array_heatmap = array_heatmap + array_val
# print(array_heatmap)
html = popup_html_flood(d[0])
popup = folium.Popup(folium.Html(html, script=True), max_width=500)
HeatMap(array_heatmap).add_to(bristol_cluster)

loc = ' HeatMap - Floodability Risk - Nationwide Mortgages '
title_html = '''
             <h2 align="center" style="font-size:24px"><b>{}</b></h2>
             '''.format(loc) 
bristol_cluster = bristol_cluster.get_root().html.add_child(folium.Element(title_html))

bristol_cluster.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Flood_Rating_heatmap.html')

In [38]:
#Built Types

import folium
from folium.features import CustomIcon

bristol_buildType =folium.Map(location=[lat,lon],zoom_start=10)

#List of Houses
df_house = df_nationwide_loanbook.loc[df_nationwide_loanbook['PROPERTY_TYPE'] == 'House']

for d in df_house.iterrows():
        html = popup_html_flood(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        house_icon = folium.features.CustomIcon('icons/house.png', icon_size=(20, 20))
        
        #colour_d1point = FloodRatingSwitch(d[1]['PROPERTY_TYPE'], d[1]['PROPERTY_TYPE'])

        folium.Marker(location=[d[1]['LATITUDE'], d[1]['LONGITUDE']],
                                    icon=house_icon,
                                    popup=popup
                     ).add_to(bristol_buildType)

#List of Flats

df_flat = df_nationwide_loanbook.loc[df_nationwide_loanbook['PROPERTY_TYPE'] == 'Flat']

for d in df_flat.iterrows():
        html = popup_html_flood(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        flat_icon = folium.features.CustomIcon('icons/flat.png', icon_size=(20, 20))
        
        folium.Marker(location=[d[1]['LATITUDE'], d[1]['LONGITUDE']],
                                    icon=flat_icon,
                                    popup=popup
                     ).add_to(bristol_buildType)


df_bungalow = df_nationwide_loanbook.loc[df_nationwide_loanbook['PROPERTY_TYPE'] == 'Bungalow']

#List of Bungalows

for d in df_bungalow.iterrows():
        html = popup_html_flood(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)        
        bungalow_icon = folium.features.CustomIcon('icons/bungalow.png', icon_size=(20, 20))
        
        folium.Marker(location=[d[1]['LATITUDE'], d[1]['LONGITUDE']],
                                    icon=bungalow_icon,
                                    popup=popup
                     ).add_to(bristol_buildType)

#List of Maisonettes

df_maisonette = df_nationwide_loanbook.loc[df_nationwide_loanbook['PROPERTY_TYPE'] == 'Maisonette']      
 
for d in df_maisonette.iterrows():
        html = popup_html_flood(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)        
        maisonette_icon = folium.features.CustomIcon('icons/maisonette.png', icon_size=(20, 20))
        
        folium.Marker(location=[d[1]['LATITUDE'], d[1]['LONGITUDE']],
                                    icon=maisonette_icon,
                                    popup=popup
                     ).add_to(bristol_buildType)


loc = ' Geo-Location Map - Property-Types '
title_html = '''
             <h2 align="center" style="font-size:24px"><b>{}</b></h2>
             '''.format(loc) 
bristol_buildType = bristol_buildType.get_root().html.add_child(folium.Element(title_html))


bristol_buildType.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Flood_Rating_builtTypes.html')

In [39]:
bristol_airbus_nwide_loanbook_mcluster=folium.Map(location=[lat,lon],zoom_start=10)
from folium.plugins import MarkerCluster

cities=['Camden', 'Bristol, City of', 'Bath and North East Somerset', 'North Somerset', 'Tower Hamlets', 'South Gloucestershire', 'Wiltshire']


    
def add_marker(city):
    
    if city == cities[0]:
        mCluster_camden.add_child(marker)
    elif city == cities[1]:
        mCluster_bristol.add_child(marker)
    elif city == cities[2]:
        mCluster_bath.add_child(marker)
    elif city == cities[3]:
        mCluster_somerset.add_child(marker)
    elif city == cities[4]:
        mCluster_hamlets.add_child(marker)
    elif city == cities[5]:
        mCluster_south.add_child(marker)
    elif city == cities[6]:
        mCluster_wiltshire.add_child(marker)
    return bristol_airbus_nwide_loanbook_mcluster

In [40]:
bristol_airbus_nwide_loanbook_mcluster=folium.Map(location=[lat,lon],zoom_start=10)
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

mCluster_camden = MarkerCluster(name = cities[0]).add_to(bristol_airbus_nwide_loanbook_mcluster)
mCluster_bristol = MarkerCluster(name = cities[1]).add_to(bristol_airbus_nwide_loanbook_mcluster)
mCluster_bath = MarkerCluster(name = cities[2]).add_to(bristol_airbus_nwide_loanbook_mcluster)
mCluster_somerset = MarkerCluster(name = cities[3]).add_to(bristol_airbus_nwide_loanbook_mcluster)
mCluster_hamlets = MarkerCluster(name = cities[4]).add_to(bristol_airbus_nwide_loanbook_mcluster)
mCluster_south = MarkerCluster(name = cities[5]).add_to(bristol_airbus_nwide_loanbook_mcluster)
mCluster_wiltshire = MarkerCluster(name = cities[6]).add_to(bristol_airbus_nwide_loanbook_mcluster)

for d in df_nationwide_loanbook.iterrows():
    location = d[1]['LATITUDE'], d[1]['LONGITUDE']
    html = popup_html_flood(d[0])
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    marker = folium.Marker(location=location, popup=popup)
    city = d[1]['LOCAL_AUTHORITY_LABEL']
    bristol_airbus_nwide_loanbook_mcluster = add_marker(city)
        
folium.LayerControl().add_to(bristol_airbus_nwide_loanbook_mcluster);

loc = ' Search by County - Property Floodability '
title_html = '''
             <h2 align="center" style="font-size:24px"><b>{}</b></h2>
             '''.format(loc) 
bristol_airbus_nwide_loanbook_mcluster = bristol_airbus_nwide_loanbook_mcluster.get_root().html.add_child(folium.Element(title_html))


bristol_airbus_nwide_loanbook_mcluster.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Flood_Rating_multicluster.html')

In [41]:
exit()

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
